$\textbf{Short Discription:}$  
This Jupyter notebook primarily focuses on the implementation of Unet model and its application from scratch on the Image Segmentation problem. Training was performed using the CWFID dataset and 

In the 'Image Classification with U-Net' folder, Unet model was applied on image segmentation with the CWFID dataset. Given the limited size of the CWFID dataset, this experiment is tried to enhance performance through Transfer Learning. To achieve this, I defined a classification model that is identical to the first half of the Unet model. This model was trained on the 'Deep Weeds' dataset using cross-entropy loss.

Following that, I fine-tuned the U-Net model using the CWFID dataset. During this fine-tuning process, all layers were kept frozen with pretrained weights, and only the second half of the model were trained, finally fine-tuned the entire model on the CWFID dataset.



# Import Libraries

In [1]:
import os
import yaml
import numpy as np
np.random.seed(1)
from skimage.io import imread
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(1)
from tensorflow.keras.preprocessing.image import load_img,img_to_array
#from google.colab import files
import tensorflow_datasets as tfds
from PIL import Image

d:\software\python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

There are two dataset used in this experiment, "Deep Weeds" and "CWFID", where the "Deep Weeds" is used for pretraining the custome classification model, as these two datasets are similar (Deep weeds is available in tfds-nightly)

In [6]:
def load_deepweeds(reduce_num=None):
  train_x = []
  train_y = []
  ds = tfds.load('deep_weeds', split='train', as_supervised=True)
  for image,label in tfds.as_numpy(ds):
    train_x.append(np.array(Image.fromarray(image).resize((224,224))))
    train_y.append(label)
  if reduce_num:
    train_x = train_x[:reduce_num]
    train_y = train_y[:reduce_num]
  train_x = np.array(train_x)/255.0
  train_y = tf.keras.utils.to_categorical(np.array(train_y))
  return train_x, train_y


def load_segmentation_data():
  data = np.load('segmentation_data.npz')
  train_x = data['train_x']
  train_y = data['train_y']
  test_x = data['test_x']
  test_y = data['test_y']
  return train_x, train_y, test_x, test_y

train_x2, train_y2 = load_deepweeds(reduce_num=1000)
train_x, train_y, test_x, test_y = load_segmentation_data()

# Models architecture and Loss function

In [7]:
# cross-entropy loss at individual spatial positions, averaged over positions
@tf.function
def mycrossentropy(y_true, y_pred):
    losses = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred, axis=3) # compute crossentropy loss from logits, for each spatial position
    return tf.math.reduce_mean(losses,axis=[1,2]) # average out the two axes corresponding to spatial position

In [9]:
def classification_model():
  inputs = tf.keras.layers.Input((224,224,3))
  x = inputs
  x224 = tf.keras.layers.Conv2D(16,3,padding='same',activation='relu',strides=1,name='L224')(inputs)
  x112 = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu',strides=2,name='L112a')(x224)
  x112 = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu',strides=1,name='L112')(x112)
  x56 = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',strides=2,name='L56a')(x112)
  x56 = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',strides=1,name='L56')(x56)
  x28 = tf.keras.layers.Conv2D(128,3,padding='same',activation='relu',strides=2,name='L28a')(x56)
  x28 = tf.keras.layers.Conv2D(128,3,padding='same',activation='relu',strides=1,name='L28')(x28)
  x14 = tf.keras.layers.Conv2D(256,3,padding='same',activation='relu',strides=2,name='L14a')(x28)
  x14 = tf.keras.layers.Conv2D(256,3,padding='same',activation='relu',strides=1,name='L14')(x14)
  x = tf.keras.layers.GlobalAveragePooling2D()(x14)
  outputs = tf.keras.layers.Dense(9,activation='softmax')(x)
  model = tf.keras.Model(inputs=inputs,outputs=outputs)
  model.summary()
  return model

def unet_model():
  inputs = tf.keras.layers.Input((224,224,3))
  x = inputs
  x224 = tf.keras.layers.Conv2D(16,3,padding='same',activation='relu',strides=1,name='L224')(inputs)
  x112 = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu',strides=2,name='L112a')(x224)
  x112 = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu',strides=1,name='L112')(x112)
  x56 = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',strides=2,name='L56a')(x112)
  x56 = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',strides=1,name='L56')(x56)
  x28 = tf.keras.layers.Conv2D(128,3,padding='same',activation='relu',strides=2,name='L28a')(x56)
  x28 = tf.keras.layers.Conv2D(128,3,padding='same',activation='relu',strides=1,name='L28')(x28)
  x14 = tf.keras.layers.Conv2D(256,3,padding='same',activation='relu',strides=2,name='L14a')(x28)
  x14 = tf.keras.layers.Conv2D(256,3,padding='same',activation='relu',strides=1,name='L14')(x14)
  x = tf.keras.layers.Conv2DTranspose(128,3,strides=2,padding='same',activation='relu')(x14)
  x = tf.keras.layers.Conv2D(128,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,x28])
  x = tf.keras.layers.Conv2DTranspose(64,3,strides=2,padding='same',activation='relu')(x)
  x = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,x56])
  x = tf.keras.layers.Conv2DTranspose(32,3,strides=2,padding='same',activation='relu')(x)
  x = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,x112])
  x = tf.keras.layers.Conv2DTranspose(16,3,strides=2,padding='same',activation='relu')(x)
  x = tf.keras.layers.Conv2D(16,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,x224])
  outputs = tf.keras.layers.Conv2D(3,1,padding='same')(x)
  unet = tf.keras.Model(inputs=inputs,outputs=outputs)
  unet.summary()
  return unet



def unet_model_transfer(model):
  inputs = model.input
  x = model.get_layer('L14').output

  x = tf.keras.layers.Conv2DTranspose(128, 3, strides=(2,2), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(128,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,model.get_layer('L28').output])

  x = tf.keras.layers.Conv2DTranspose(64, 3, strides=(2,2), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,model.get_layer('L56').output])

  x = tf.keras.layers.Conv2DTranspose(32, 3, strides=(2,2), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,model.get_layer('L112').output])

  x = tf.keras.layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(16,3,padding='same',activation='relu',strides=1)(x)
  x = tf.keras.layers.Concatenate()([x,model.get_layer('L224').output])

  outputs = tf.keras.layers.Conv2D(3, 1, padding='same')(x)

  unet = tf.keras.Model(inputs=model.inputs,outputs=outputs)
  unet.summary()
  return unet

In [10]:
def save_predictions_as_images(predictions,test_y):
    pred_y = np.zeros(predictions.shape)
    # convert class scores to one-hot encoded predictions
    for i in range(0,test_x.shape[0]):
        for x in range(0,test_x.shape[1]):
            for y in range(0,test_x.shape[2]):
                pred_y[i,x,y,np.argmax(predictions[i,x,y,:])]=1
    # plot the three classes as RGB image 
    for i in range(0,test_x.shape[0]):
        Image.fromarray((test_y[i,:,:,:]*255).astype('uint8')).save('%i_true.png' % i)
        Image.fromarray((pred_y[i,:,:,:]*255).astype('uint8')).save('%i_pred.png' % i)

# Training 

Due to the computational limitation, the dataset size and the training epochs were reduced, which led to not ideal performance. Additionally, the predition images and original images are saved in the folder "predition images"

In [13]:
running_epochs = 20 # 500

unet = unet_model()
unet.compile(loss=mycrossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy']) 
unet.fit(train_x, train_y, batch_size = 10, epochs = running_epochs, validation_data=(test_x, test_y))

predictions = unet.predict(test_x)
save_predictions_as_images(predictions,test_y)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 L224 (Conv2D)                  (None, 224, 224, 16  448         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 L112a (Conv2D)                 (None, 112, 112, 32  4640        ['L224[0][0]']                   
                                )                                                           

In [15]:
model = classification_model()
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(train_x2,train_y2,epochs=25)

unet = unet_model_transfer(model)

# first, only train last layer
for layer in unet.layers:
  if "L" in layer.name:
    print("Setting layer %s to not trainable" % layer.name)
    layer.trainable = False
unet.compile(loss=mycrossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy']) 
unet.fit(train_x, train_y, batch_size = 10, epochs = running_epochs,validation_data=(test_x, test_y))

# finally train all layers
for layer in unet.layers:
    print("Setting layer %s to trainable" % layer.name)
    layer.trainable = True
unet.compile(loss=mycrossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy']) 
unet.fit(train_x, train_y, batch_size = 10, epochs = running_epochs,validation_data=(test_x, test_y))

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 L224 (Conv2D)               (None, 224, 224, 16)      448       
                                                                 
 L112a (Conv2D)              (None, 112, 112, 32)      4640      
                                                                 
 L112 (Conv2D)               (None, 112, 112, 32)      9248      
                                                                 
 L56a (Conv2D)               (None, 56, 56, 64)        18496     
                                                                 
 L56 (Conv2D)                (None, 56, 56, 64)        36928     
                                                                 
 L28a (Conv2D)               (None, 28, 28, 128)       7385